In [18]:
!pip uninstall -y kaggle
!pip install --upgrade pip
!pip install kaggle==1.5.6

Found existing installation: kaggle 1.5.6
Uninstalling kaggle-1.5.6:
  Successfully uninstalled kaggle-1.5.6
Requirement already up-to-date: pip in /usr/local/lib/python3.6/dist-packages (20.1.1)
Processing /root/.cache/pip/wheels/01/3e/ff/77407ebac3ef71a79b9166a8382aecf88415a0bcbe3c095a01/kaggle-1.5.6-py3-none-any.whl


In [19]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
!kaggle -v

Kaggle API 1.5.6


In [20]:
!kaggle competitions download  -c defense-project


defense-project.zip: Skipping, found more recently modified local copy (use --force to force download)


In [21]:

!unzip defense-project.zip

Archive:  defense-project.zip
replace submissions_sample.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [298]:
import torch
import torch.nn.functional as F
import torch.optim as optim
import torch.nn as nn

import numpy as np
import sys

import pandas as pd

import random

from sklearn.preprocessing import StandardScaler

device = 'cuda' if torch.cuda.is_available() else 'cpu'

random.seed(484)
torch.manual_seed(484)
if device == 'cuda':
  torch.cuda.manual_seed_all(484)

In [299]:
train = pd.read_csv('train_data.csv')
train

,# mean_0_a,mean_1_a,mean_2_a,mean_3_a,mean_4_a,mean_d_0_a,mean_d_1_a,mean_d_2_a,mean_d_3_a,mean_d_4_a,mean_d_0_a2,mean_d_1_a2,mean_d_2_a2,mean_d_3_a2,mean_d_4_a2,mean_d_5_a,mean_d_6_a,mean_d_7_a,mean_d_8_a,mean_d_9_a,mean_d_10_a,mean_d_11_a,mean_d_12_a,mean_d_13_a,mean_d_14_a,mean_d_15_a,mean_d_16_a,mean_d_17_a,mean_d_18_a,mean_d_19_a,mean_d_20_a,mean_d_21_a,mean_d_22_a,mean_d_23_a,mean_d_24_a,mean_d_25_a,mean_d_26_a,mean_d_27_a,mean_d_28_a,mean_d_29_a,...,fft_711_b,fft_712_b,fft_713_b,fft_714_b,fft_715_b,fft_716_b,fft_717_b,fft_718_b,fft_719_b,fft_720_b,fft_721_b,fft_722_b,fft_723_b,fft_724_b,fft_725_b,fft_726_b,fft_727_b,fft_728_b,fft_729_b,fft_730_b,fft_731_b,fft_732_b,fft_733_b,fft_734_b,fft_735_b,fft_736_b,fft_737_b,fft_738_b,fft_739_b,fft_740_b,fft_741_b,fft_742_b,fft_743_b,fft_744_b,fft_745_b,fft_746_b,fft_747_b,fft_748_b,fft_749_b,label
0,26.700000,31.000000,34.500000,24.217760,26.700000,-3.470000,-0.170,-2.555242,-1.4400,6.315870,25.400000,30.50,38.100000,24.300000,26.700000,31.400000,31.70000,33.500000,25.600000,20.400000,28.400000,31.90000,32.500000,26.400000,30.400000,21.591346,30.000000,34.000000,20.673048,29.300000,-5.990000,-1.165926,4.600000,-1.361479,6.240000,-2.980000,-1.406815,5.600000,-2.160000,-3.781638,...,8.720546,16.734340,16.728144,8.730000,141.000000,4.210000,18.400000,18.42176,4.210,193.00000,-15.700000,32.300000,32.3,-15.700000,154.000000,-0.743000,20.100000,20.100000,-0.743000,110.000000,-28.242850,66.100000,66.100000,-28.210441,190.950627,-13.900000,26.800000,26.800000,-13.900000,199.000000,-29.300000,26.800000,26.8,-29.300000,193.000000,-4.11,-17.174885,-17.100000,-4.110000,0
1,28.800000,32.700000,35.000000,26.393804,24.400000,1.870000,1.650,8.770000,1.2400,2.580000,29.100000,31.80,32.000000,25.800000,18.088154,26.700000,31.90000,29.200000,25.677609,28.100000,29.500000,32.40000,39.600000,26.700000,24.000000,30.000000,34.600000,39.200000,27.300000,27.300000,2.340000,-0.081700,2.820000,0.094000,-10.000000,-0.433000,-0.620969,-7.500000,-0.891000,-5.990000,...,28.000000,-31.300000,-31.300203,27.980281,168.000000,9.620000,-28.600000,-28.60000,9.620,208.00000,19.300000,-30.000000,-30.0,19.300000,94.726129,23.300000,-9.149943,-9.110000,23.322822,201.000000,14.300000,-21.000000,-21.000000,14.300000,145.000000,-6.020000,17.965147,18.014800,-6.020000,101.000000,25.900000,-2.900000,-2.9,25.905983,69.441984,-7.10,39.200000,39.200000,-7.100000,0
2,15.667977,25.900000,-141.000000,15.258776,26.200000,-7.354916,-2.350,3.715261,-6.2900,7.600000,21.300000,27.90,-163.638147,20.900000,26.700000,16.600000,26.20000,-124.000000,15.500000,18.175889,11.700000,24.30000,-153.000000,14.200000,30.100000,13.300000,24.512478,-124.000000,9.824515,30.000000,4.770000,1.700000,-39.000000,5.340000,8.380000,9.570000,3.640000,-9.920000,7.397449,-3.440000,...,13.800000,2.890000,2.672459,13.800000,391.000000,-236.000000,72.800000,72.80000,-236.000,-623.00000,505.000000,-225.000000,-225.0,505.000000,-115.000000,202.000000,-58.389994,-58.798806,202.000000,343.000000,-165.455388,50.600000,50.600000,-166.000000,-759.000000,646.701939,-282.000000,-282.000000,646.000000,47.100000,80.800000,-33.100000,-33.1,80.800000,346.000000,-219.00,108.230070,109.000000,-219.000000,2
3,30.900000,32.000000,31.800000,29.000000,25.600000,-4.571665,-1.430,-1.220000,-2.3900,1.490000,32.800000,30.90,30.900000,29.800000,23.900000,33.600000,34.50000,34.000000,30.725413,25.900000,29.300000,30.20000,29.400000,29.600000,29.200000,27.900000,32.400000,33.000000,26.144932,23.600000,-0.845000,-3.650000,-3.070000,-0.891000,-1.970000,3.430000,0.686000,1.450000,0.264587,-5.300992,...,-13.100000,33.900000,33.900000,-13.100000,164.000000,0.298000,0.738000,0.73800,0.298,181.00000,3.570000,-29.900000,-29.9,3.570000,151.000000,-4.753291,-3.312440,-3.280000,-4.770000,141.042113,-23.700000,10.100000,10.100000,-23.779761,137.000000,-2.700000,-2.311960,-2.376683,-2.700000,106.943135,-19.100000,26.600000,26.6,-19.100000,215.000000,14.40,-43.500000,-43.500000,14.400000,0
4,-3.190000

In [300]:


scaler = StandardScaler()

x_train_data = train.loc[:,'# mean_0_a':'fft_749_b']
y_train_data = train.loc[:,'label']

xs_data = scaler.fit_transform(x_train_data)

In [301]:
x_train = torch.FloatTensor(xs_data[:1200])
x_vali = torch.FloatTensor(xs_data[1200:]) # Validation Data
y_train = torch.LongTensor(y_train_data[:1200].values)
y_vali = torch.LongTensor(y_train_data[1200:].values)

In [302]:
print(x_train.shape)
print(x_vali.shape)

torch.Size([1200, 2548])
torch.Size([100, 2548])


In [303]:
l1 = torch.nn.Linear(2548,128) #딥러닝 모델로 수정
l2 = torch.nn.Linear(128, 128)
l3 = torch.nn.Linear(128, 128)
l4 = torch.nn.Linear(128, 128)
l5 = torch.nn.Linear(128, 3)

dr = torch.nn.Dropout(0.4)
relu = torch.nn.LeakyReLU()


In [305]:
torch.nn.init.xavier_uniform_(l1.weight)
torch.nn.init.xavier_uniform_(l2.weight)
torch.nn.init.xavier_uniform_(l3.weight)
torch.nn.init.xavier_uniform_(l4.weight)
torch.nn.init.xavier_uniform_(l5.weight)

model = torch.nn.Sequential(
    l1, relu, dr, 
    l2, relu, dr, 
    l3, relu, dr, 
#    l4, relu, dr, 
    l5
).to(device)
model

Sequential(
  (0): Linear(in_features=2548, out_features=128, bias=True)
  (1): LeakyReLU(negative_slope=0.01)
  (2): Dropout(p=0.4, inplace=False)
  (3): Linear(in_features=128, out_features=128, bias=True)
  (4): LeakyReLU(negative_slope=0.01)
  (5): Dropout(p=0.4, inplace=False)
  (6): Linear(in_features=128, out_features=128, bias=True)
  (7): LeakyReLU(negative_slope=0.01)
  (8): Dropout(p=0.4, inplace=False)
  (9): Linear(in_features=128, out_features=3, bias=True)
)

In [306]:
train_dataset = torch.utils.data.TensorDataset(x_train, y_train)
data_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                          batch_size=100,
                                          shuffle=True,
                                          drop_last=False)

In [307]:
loss = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001) 

In [308]:
total_batch = len(data_loader)
model_history = []
err_history = []
vali_err_history = []
x_vali = x_vali.to(device)
y_vali = y_vali.to(device)
for epoch in range(1, 1+300):
    avg_cost = 0

    for X, Y in data_loader:

        X = X.to(device)
        Y = Y.to(device)

        # 그래디언트 초기화
        optimizer.zero_grad()
        # Forward 계산
        hypothesis = model(X)
        # Error 계산
        cost = loss(hypothesis, Y)
        # Backparopagation
        cost.backward()
        # 가중치 갱신
        optimizer.step()

        # 평균 Error 계산
        avg_cost += cost
    avg_cost /= total_batch
    
    with torch.no_grad():
        pred = model(x_vali)
        torch.nn.CrossEntropyLoss().to(device)
        vali_err = loss(pred, y_vali)
        vali_err_history.append(vali_err)
    
    model_history.append(model)
    err_history.append(avg_cost)

    if epoch % 50 == 1 :
        print('Epoch:', '%04d' % (epoch), 'cost =', '{:.9f}'.format(avg_cost))

print('Epoch:', '%04d' % (epoch), 'cost =', '{:.9f}'.format(avg_cost))

Epoch: 0001 cost = 1.156915069
Epoch: 0051 cost = 0.128378332
Epoch: 0101 cost = 0.066099733
Epoch: 0151 cost = 0.037737634
Epoch: 0201 cost = 0.027432991
Epoch: 0251 cost = 0.015230301
Epoch: 0300 cost = 0.021553837


In [309]:
test = pd.read_csv('test_data.csv')
test = test.to_numpy()
test = scaler.transform(test)
with torch.no_grad():
    x_test = torch.FloatTensor(test).to(device)
    
    pred = model(x_test)
    pred = pred.cpu()
    
pred

tensor([[ 5.0828, -1.9217, -4.8278],
        [-5.2810,  2.9427,  1.1963],
        [-7.8129, -4.5087,  7.9448],
        ...,
        [-4.6850,  1.9899,  2.5810],
        [-5.6334, -3.0838,  3.4624],
        [-8.2430, -7.2243, 11.7334]])

In [310]:
real_test_df = pd.DataFrame([[i, r] for i, r in enumerate(torch.argmax(pred, dim=1).numpy())], columns=['Id',  'Category'])
real_test_df.to_csv('result.csv', mode='w', index=False)

In [311]:
real_test_df

,Id,Category
0,0,0
1,1,1
2,2,2
3,3,0
4,4,1
...,...,...
827,827,2
828,828,1
829,829,2
830,830,2


In [312]:
!kaggle competitions submit -c defense-project -f result.csv -m "4layer 128  에폭 300 cost = 0.010208621"

100% 4.78k/4.78k [00:00<00:00, 21.7kB/s]
Successfully submitted to 영상물 시청시 뇌파에 대한 감정분류